In [ ]:
# file: noise_ceiling_demo.py
# Purpose: Demonstrate label-noise accuracy ceiling (1 - rho)
# Mục đích: Minh họa trần độ chính xác khi có nhiễu nhãn (1 - rho)

import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
rng = np.random.default_rng(42) #initiate random number generator
# rng = np.random.default_rng(42) # khởi tạo bộ tạo số ngẫu nhiên

def flip_labels(y_true, pi=None, alpha=0.0, beta=0.0, rng=None):
    """
    Flip positives with prob alpha, negatives with prob beta.
    Returns noisy labels tilde_y and the realized corruption rate rho_hat.
    """
    # Lật nhãn dương với xác suất alpha, nhãn âm với xác suất beta.
    # Trả về nhãn nhiễu tilde_y và tỉ lệ nhiễu thực tế rho_hat.
    if rng is None:
        rng = np.random.default_rng()
    y_true = y_true.astype(int)
    flip = np.zeros_like(y_true, dtype=bool) # Mảng boolean để đánh dấu các nhãn bị lật

    pos_idx = (y_true == 1) # Chỉ mục của các nhãn dương (lớp 1)
    neg_idx = ~pos_idx      # Chỉ mục của các nhãn âm (lớp 0)

    # Xác định nhãn nào sẽ bị lật dựa trên xác suất alpha và beta
    flip[pos_idx] = rng.random(np.sum(pos_idx)) < alpha
    flip[neg_idx] = rng.random(np.sum(neg_idx)) < beta

    y_noisy = y_true.copy() # Tạo bản sao của nhãn gốc
    y_noisy[flip] = 1 - y_noisy[flip] # Lật các nhãn được đánh dấu trong mảng flip
    rho_hat = np.mean(flip) # Tính tỉ lệ nhãn bị lật thực tế (empirical noise rate)
    return y_noisy, rho_hat

In [ ]:
# Sử dụng hàm flip_labels để tạo nhãn nhiễu với alpha=0.2 và beta=0.2
y_noisy, rho = flip_labels(y_true, alpha=0.2, beta=0.2)
print(rho)  # kiểm tra tỉ lệ nhãn bị lật (check the empirical noise rate)

0.20071


In [ ]:
# 1) Make a clean, separable-ish dataset
X, y_true = make_classification(
    n_samples=100_000, n_features=20, n_informative=8, n_redundant=2,
    weights=[0.5, 0.5], class_sep=1.5, random_state=7
)
pi_hat = np.mean(y_true)  # empirical prior ~ 0.5

In [ ]:
# 2) Define noise: ONLY positives flip at rate alpha=0.05 (beta=0)
alpha, beta = 0.05, 0.0
y_noisy, rho_hat = flip_labels(y_true, alpha=alpha, beta=beta, rng=rng)

# The theoretical ceiling: 1 - rho, rho = pi*alpha + (1-pi)*beta
rho_theory = pi_hat * alpha + (1 - pi_hat) * beta
ceiling = 1.0 - rho_theory

print(f"Empirical class prior pi ≈ {pi_hat:.3f}")
print(f"α={alpha:.2%}, β={beta:.2%} → theoretical ρ = {rho_theory:.3%}")
print(f"Accuracy ceiling vs noisy labels (1 - ρ) ≈ {ceiling:.3%}")

Empirical class prior pi ≈ 0.500
α=5.00%, β=0.00% → theoretical ρ = 2.502%
Accuracy ceiling vs noisy labels (1 - ρ) ≈ 97.498%


# Minh họa Trần Độ Chính xác với Nhiễu Nhãn (Label Noise Accuracy Ceiling)

Notebook này minh họa khái niệm "trần độ chính xác" (accuracy ceiling) khi dữ liệu huấn luyện bị nhiễu nhãn. Khi nhãn bị sai, ngay cả một mô hình hoàn hảo cũng không thể đạt được độ chính xác 100% khi đánh giá trên tập nhãn bị nhiễu đó. Trần độ chính xác này thường được tính bằng `1 - ρ`, trong đó `ρ` (rho) là tỉ lệ nhãn bị nhiễu.

Chúng ta sẽ thực hiện các bước sau:
1. Tạo một tập dữ liệu sạch.
2. Giới thiệu nhiễu vào nhãn của tập huấn luyện.
3. Tính toán trần độ chính xác lý thuyết.
4. Minh họa rằng một mô hình lý tưởng (oracle) đạt độ chính xác gần bằng trần này khi đánh giá trên nhãn nhiễu.
5. Huấn luyện một mô hình thực tế trên nhãn nhiễu và so sánh hiệu suất của nó khi đánh giá trên nhãn nhiễu và nhãn thật.

In [ ]:
# 3) Show oracle (predicts true y) evaluated against NOISY labels
# 3) Hiển thị độ chính xác của mô hình lý tưởng (oracle - dự đoán nhãn thật y) khi đánh giá trên nhãn nhiễu
acc_oracle_vs_noisy = accuracy_score(y_noisy, y_true)
# Tính độ chính xác của mô hình lý tưởng (sử dụng nhãn thật y_true làm dự đoán) so với nhãn nhiễu y_noisy
print(f"Oracle accuracy vs noisy labels: {acc_oracle_vs_noisy:.3%} (≈ ceiling)")
# In kết quả độ chính xác của mô hình lý tưởng so với nhãn nhiễu, so sánh với trần lý thuyết


Oracle accuracy vs noisy labels: 97.502% (≈ ceiling)


### Exercise 3:  Noise Simulation on MNIST
### Bài tập 3: Mô phỏng Nhiễu trên tập dữ liệu MNIST


## Exercise 1: Debugging CSVs (headers + types + dates)
## Bài tập 1: Gỡ lỗi file CSV (tiêu đề cột + kiểu dữ liệu + định dạng ngày)


In [ ]:
# amounts → float
# Chuyển đổi cột số tiền sang kiểu float
if "amount" in df: # Kiểm tra xem cột "amount" có tồn tại trong DataFrame không
    df["amount"] = (
        df["amount"].astype(str) # Chuyển cột "amount" sang kiểu chuỗi (string)
        .str.replace(r"[^0-9.\-]", "", regex=True) # Loại bỏ tất cả ký tự không phải số, dấu chấm, dấu gạch ngang (sử dụng regex)
        .replace({"": np.nan}) # Thay thế các chuỗi rỗng thành NaN (Not a Number)
        .astype(float) # Chuyển cột sang kiểu số thực (float)
    )


### Code Demo: Logging Bias
### Demo mã nguồn: Thiên kiến ghi nhật ký (Logging Bias)



### Code Demo: Data Drift
### Demo mã nguồn: Trôi dạt dữ liệu (Data Drift)


### Mini Demo: The Data Trust Pyramid
### Mini Demo: Tháp Độ Tin Cậy Dữ liệu


In [ ]:
# 4) Train/test split using NOISY labels (realistic scenario)
# 4) Chia tập huấn luyện/kiểm tra sử dụng nhãn nhiễu (kịch bản thực tế)
X_train, X_test, y_train_noisy, y_test_noisy, _y_train_true, y_test_true = train_test_split(
    X, y_noisy, y_true, test_size=0.3, random_state=123, stratify=y_noisy
)
# Chia tập dữ liệu X, nhãn nhiễu y_noisy và nhãn thật y_true thành tập huấn luyện và tập kiểm tra
# test_size=0.3: 30% dữ liệu dùng cho tập kiểm tra
# random_state=123: Đảm bảo việc chia dữ liệu là nhất quán
# stratify=y_noisy: Đảm bảo tỷ lệ các lớp trong tập huấn luyện và kiểm tra tương tự như trong y_noisy


clf = LogisticRegression(max_iter=1000, n_jobs=-1)
# Khởi tạo mô hình Hồi quy Logistic
# max_iter=1000: Số lần lặp tối đa cho thuật toán giải
# n_jobs=-1: Sử dụng tất cả các lõi CPU khả dụng để tăng tốc độ
clf.fit(X_train, y_train_noisy)
# Huấn luyện mô hình trên tập huấn luyện sử dụng nhãn nhiễu y_train_noisy

# Evaluate vs NOISY labels (what many teams do by mistake)
# Đánh giá trên nhãn nhiễu (điều mà nhiều đội làm sai lầm)
acc_vs_noisy = accuracy_score(y_test_noisy, clf.predict(X_test))
# Tính độ chính xác của mô hình được huấn luyện khi dự đoán trên X_test và so sánh với nhãn nhiễu y_test_noisy

# Evaluate vs TRUE labels (if you had a clean gold set)
# Đánh giá trên nhãn thật (nếu bạn có một tập nhãn thật sạch - gold set)
acc_vs_true = accuracy_score(y_test_true, clf.predict(X_test))
# Tính độ chính xác của mô hình được huấn luyện khi dự đoán trên X_test và so sánh với nhãn thật y_test_true

print(f"Model accuracy vs NOISY labels: {acc_vs_noisy:.3%} (capped near ceiling)")
# In độ chính xác của mô hình so với nhãn nhiễu (thường bị giới hạn gần trần độ chính xác)
print(f"Model accuracy vs TRUE  labels: {acc_vs_true:.3%} (can exceed ceiling)")
# In độ chính xác của mô hình so với nhãn thật (có thể vượt qua trần độ chính xác khi đánh giá trên nhãn nhiễu)


Model accuracy vs NOISY labels: 93.497% (capped near ceiling)
Model accuracy vs TRUE  labels: 95.697% (can exceed ceiling)


In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Định nghĩa hàm run để chạy toàn bộ quy trình với các tham số nhiễu và phân tách lớp khác nhau
def run(alpha=0.05, beta=0.0, class_sep=1.5):
    # Tạo tập dữ liệu sạch tương tự như trước
    X, y_true = make_classification(
        n_samples=100_000, n_features=20, n_informative=8, n_redundant=2,
        weights=[0.5, 0.5], class_sep=class_sep, random_state=7
    )
    # Tính tỷ lệ lớp thực tế (prior)
    pi_hat = np.mean(y_true)
    # Khởi tạo bộ tạo số ngẫu nhiên
    rng = np.random.default_rng(42)

    # Định nghĩa lại hàm flip_labels bên trong hàm run để sử dụng rng cục bộ
    def flip_labels(y, alpha, beta):
        y = y.astype(int)
        pos = (y == 1) # Chỉ mục nhãn dương
        neg = ~pos      # Chỉ mục nhãn âm
        flip = np.zeros_like(y, dtype=bool) # Mảng đánh dấu các nhãn bị lật
        # Lật nhãn dương với xác suất alpha
        flip[pos] = rng.random(np.sum(pos)) < alpha
        # Lật nhãn âm với xác suất beta
        flip[neg] = rng.random(np.sum(neg)) < beta
        y_noisy = y.copy() # Bản sao nhãn gốc
        y_noisy[flip] = 1 - y_noisy[flip] # Thực hiện lật nhãn
        return y_noisy, np.mean(flip) # Trả về nhãn nhiễu và tỷ lệ nhiễu thực tế

    # Tạo nhãn nhiễu và tính tỷ lệ nhiễu thực tế
    y_noisy, rho_hat = flip_labels(y_true, alpha, beta)
    # Tính trần độ chính xác lý thuyết (1 - rho)
    ceiling = 1 - (pi_hat*alpha + (1-pi_hat)*beta)

    # In các thông số và kết quả tính toán
    print(f"pi≈{pi_hat:.3f}, alpha={alpha:.2%}, beta={beta:.2%}")
    print(f"Ceiling (1 - rho)≈{ceiling:.3%}; Empirical rho≈{rho_hat:.3%}")

    from sklearn.model_selection import train_test_split
    # Chia tập dữ liệu thành huấn luyện và kiểm tra (X, nhãn nhiễu, nhãn thật)
    Xtr, Xte, ytr_noisy, yte_noisy, _ytr_true, yte_true = train_test_split(
        X, y_noisy, y_true, test_size=0.3, random_state=123, stratify=y_noisy
    )

    # Huấn luyện mô hình Hồi quy Logistic trên nhãn nhiễu
    clf = LogisticRegression(max_iter=1000).fit(Xtr, ytr_noisy)
    # Đánh giá độ chính xác trên tập kiểm tra sử dụng nhãn nhiễu
    print("Acc vs noisy:", accuracy_score(yte_noisy, clf.predict(Xte)))
    # Đánh giá độ chính xác trên tập kiểm tra sử dụng nhãn thật
    print("Acc vs true :", accuracy_score(yte_true , clf.predict(Xte)))


# Thử với một vài cài đặt khác nhau
# Cài đặt 1: alpha=5%, beta=0%
run(alpha=0.05, beta=0.00, class_sep=1.5)
# Cài đặt 2: alpha=10%, beta=0% (nhiều nhiễu hơn ở lớp dương)
run(alpha=0.10, beta=0.00, class_sep=1.5)
# Cài đặt 3: alpha=5%, beta=5% (nhiễu đối xứng)
run(alpha=0.05, beta=0.05, class_sep=1.5)


pi≈0.500, alpha=5.00%, beta=0.00%
Ceiling (1 - rho)≈97.498%; Empirical rho≈2.498%
Acc vs noisy: 0.9349666666666666
Acc vs true : 0.9569666666666666
pi≈0.500, alpha=10.00%, beta=0.00%
Ceiling (1 - rho)≈94.996%; Empirical rho≈4.944%
Acc vs noisy: 0.9063333333333333
Acc vs true : 0.9475666666666667
pi≈0.500, alpha=5.00%, beta=5.00%
Ceiling (1 - rho)≈95.000%; Empirical rho≈5.046%
Acc vs noisy: 0.9158333333333334
Acc vs true : 0.9624


### Exercise 3:  Noise Simulation on MNIST

In [ ]:
import torch, torchvision
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

# 1) Load MNIST
# 1) Tải tập dữ liệu MNIST
train = torchvision.datasets.MNIST(root=".", train=True, download=True,
                                   transform=transforms.ToTensor()) # Tải tập huấn luyện
test  = torchvision.datasets.MNIST(root=".", train=False, download=True,
                                   transform=transforms.ToTensor())  # Tải tập kiểm tra

# 2) Corrupt 10% of training labels
# 2) Làm nhiễu 10% nhãn của tập huấn luyện
rho = 0.10 # Tỷ lệ nhiễu mong muốn
n = len(train.targets) # Tổng số mẫu trong tập huấn luyện
idx = np.random.choice(n, int(rho*n), replace=False) # Chọn ngẫu nhiên các chỉ mục để làm nhiễu
train.targets[idx] = torch.randint(0,10,(len(idx),)) # Gán nhãn ngẫu nhiên mới cho các chỉ mục đã chọn
print(f"Injected noise rate ρ={rho:.1%}") # In tỷ lệ nhiễu đã tiêm vào

# 3) Simple CNN
# 3) Mạng CNN đơn giản
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1), nn.ReLU(), # Lớp tích chập 1
            nn.Conv2d(32, 64, 3, 1), nn.ReLU(), # Lớp tích chập 2
            nn.Flatten(), # Làm phẳng đầu ra
            nn.Linear(64*24*24, 128), nn.ReLU(),  # Lớp tuyến tính (fully connected) 1
            nn.Linear(128, 10) # Lớp tuyến tính đầu ra (10 lớp cho MNIST)
        )
    def forward(self, x): return self.net(x) # Định nghĩa cách dữ liệu truyền qua mạng

# 4) Training loop (short)
# 4) Vòng lặp huấn luyện (ngắn)
device = "cuda" if torch.cuda.is_available() else "cpu" # Chọn thiết bị (GPU hoặc CPU)
model = CNN().to(device) # Khởi tạo mô hình và chuyển sang thiết bị đã chọn
opt = optim.Adam(model.parameters(), lr=1e-3) # Khởi tạo trình tối ưu hóa Adam
loss_fn = nn.CrossEntropyLoss() # Định nghĩa hàm mất mát Cross-Entropy

loader = DataLoader(train,batch_size=256,shuffle=True) # Tạo DataLoader cho tập huấn luyện (sử dụng nhãn nhiễu)
for epoch in range(1,4): # Huấn luyện trong 3 epoch
    for X,y in loader: # Duyệt qua từng batch dữ liệu
        X,y = X.to(device), y.to(device) # Chuyển dữ liệu và nhãn sang thiết bị đã chọn
        opt.zero_grad(); loss_fn(model(X),y).backward(); opt.step() # Zero gradient, tính loss, backprop, cập nhật trọng số
    print(f"Epoch {epoch} done") # In thông báo kết thúc mỗi epoch

# 5) Evaluate
# 5) Đánh giá
testloader = DataLoader(test,batch_size=512) # Tạo DataLoader cho tập kiểm tra (sử dụng nhãn thật)
def accuracy(loader): # Hàm tính độ chính xác
    correct,total=0,0 # Khởi tạo biến đếm đúng và tổng số mẫu
    for X,y in loader: # Duyệt qua từng batch dữ liệu kiểm tra
        X,y=X.to(device),y.to(device) # Chuyển dữ liệu và nhãn sang thiết bị
        pred=model(X).argmax(1) # Dự đoán lớp có xác suất cao nhất
        correct+= (pred==y).sum().item() # Đếm số dự đoán đúng
        total+= y.size(0) # Cộng tổng số mẫu trong batch
    return correct/total # Trả về độ chính xác
acc = accuracy(testloader) # Tính độ chính xác trên tập kiểm tra (sử dụng nhãn thật)
print(f"Test accuracy: {acc:.3%}, Theory ceiling: {(1-rho):.3%}") # In độ chính xác trên tập kiểm tra và trần lý thuyết (1 - tỷ lệ nhiễu)


100%|██████████| 9.91M/9.91M [00:00<00:00, 22.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 608kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 5.59MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.57MB/s]


Injected noise rate ρ=10.0%
Epoch 1 done
Epoch 2 done
Epoch 3 done
Test accuracy: 98.510%, Theory ceiling: 90.000%


## Exercise 1: Debugging CSVs (headers + types + dates)

In [ ]:
# --- install (Colab usually has pandas, but just in case)
# --- cài đặt (Colab thường đã có pandas, nhưng cài đặt đề phòng)
!pip -q install pandas # Cài đặt thư viện pandas (chế độ im lặng - không hiển thị nhiều output)

import pandas as pd, numpy as np # Import thư viện pandas và numpy
from hashlib import md5 # Import hàm md5 để tạo hash
from pathlib import Path # Import Path để làm việc với đường dẫn file


In [ ]:
BASE = Path(".")  # current working dir in Colab
# BASE = Path(".")  # thư mục làm việc hiện tại trong Colab
dirty_path = BASE/"dirty.csv" # Đường dẫn đến file CSV bị lỗi
clean_path = BASE/"clean.csv" # Đường dẫn để lưu file CSV đã làm sạch
bad_rows_path = BASE/"bad_rows.csv" # Đường dẫn để lưu các dòng dữ liệu bị lỗi


In [ ]:
# ---- create a messy CSV
# ---- tạo một file CSV lộn xộn
raw = [
    [101, "03/10/2025",              "$12.99",         "ok"],
    [102, "2025-03-11 10:30",        "€15.00",         "euro symbol, dot decimal"], # Ký hiệu Euro, dấu chấm thập phân
    [103, "10 Mar 2025 14:00 UTC",   "Rs 1,234.50",    "comma thousands"], # Dấu phẩy phân cách hàng nghìn
    [104, "03-12-25",                "ninety",         "text should become NaN"], # Văn bản đáng lẽ phải trở thành NaN (Not a Number)
    [105, "2025/03/14 16:00 +05:30", "$-9.99",         "tz offset example"], # Ví dụ về múi giờ (timezone offset)
]
# Định nghĩa dữ liệu thô dưới dạng danh sách các danh sách (list of lists)


In [ ]:
dirty = pd.DataFrame(raw, columns=["User ID"," signupdate ","amount($)","notes"])
# Tạo DataFrame từ dữ liệu thô và đặt tên cột ban đầu
dirty.to_csv(dirty_path, index=False)
# Lưu DataFrame vào file CSV bị lỗi (dirty.csv), không bao gồm chỉ mục (index) của DataFrame


In [ ]:
# ---- cleaning
# ---- quá trình làm sạch
alias = {
    "user id":"user_id", "userid":"user_id",
    " signupdate ":"signup_date","signupdate":"signup_date",
    "amount($)":"amount","amt":"amount",
}
# Định nghĩa một từ điển ánh xạ (alias) để chuẩn hóa tên cột


In [ ]:
# Hàm chuẩn hóa tên cột: loại bỏ khoảng trắng thừa, chuyển về chữ thường và áp dụng alias
def canon(c):
    c2 = c.strip().lower() # Loại bỏ khoảng trắng ở đầu/cuối và chuyển về chữ thường
    return alias.get(c2, c2) # Trả về tên chuẩn hóa từ alias hoặc tên gốc nếu không tìm thấy

df = pd.read_csv(dirty_path) # Đọc dữ liệu từ file CSV bị lỗi vào DataFrame
df.columns = [canon(c) for c in df.columns] # Áp dụng hàm canon để chuẩn hóa tên các cột của DataFrame


In [ ]:
# dates → UTC
# Chuyển đổi cột ngày tháng về định dạng UTC
df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce", utc=True)
# Sử dụng pd.to_datetime để chuyển cột "signup_date" sang kiểu datetime
# errors="coerce": Thay thế các giá trị không thể chuyển đổi thành datetime bằng NaT (Not a Time)
# utc=True: Chuyển đổi ngày giờ sang múi giờ UTC


In [ ]:
# amounts → float
if "amount" in df:
    df["amount"] = (
        df["amount"].astype(str)
        .str.replace(r"[^0-9.\-]", "", regex=True)
        .replace({"": np.nan})
        .astype(float)
    )

In [ ]:
# report + bad rows
# Báo cáo và các dòng dữ liệu bị lỗi
problems = df.isna().sum() # Đếm số lượng giá trị NaN (thiếu) trong mỗi cột
bad_rows = df[df.isna().any(axis=1)] # Lọc các dòng có ít nhất một giá trị NaN
if not bad_rows.empty: # Nếu có các dòng bị lỗi
    bad_rows.to_csv(bad_rows_path, index=False) # Lưu các dòng bị lỗi vào file CSV riêng, không bao gồm chỉ mục


In [ ]:
# schema hash
# Tạo hash cho schema (cấu trúc dữ liệu)
sig = "|".join([f"{c}:{str(dt)}" for c, dt in zip(df.columns, df.dtypes)])
# Tạo một chuỗi signature bằng cách nối tên cột và kiểu dữ liệu của chúng, phân tách bằng "|".
# Ví dụ: "user_id:int64|signup_date:datetime64[ns, UTC]|amount:float64|notes:object|amount_num:float64"
schema_hash = md5(sig.encode()).hexdigest()
# Tính giá trị hash MD5 của chuỗi signature sau khi mã hóa sang bytes và lấy dạng hex


In [ ]:
# save
# Lưu file
df.to_csv(clean_path, index=False) # Lưu DataFrame đã làm sạch vào file CSV, không bao gồm chỉ mục

print("Nulls per column:\n", problems) # In ra số lượng giá trị NaN trong mỗi cột
print("Schema signature:", schema_hash) # In ra giá trị hash của schema
# In thông báo đã lưu các file nào (bao gồm file bad_rows.csv nếu có dòng bị lỗi)
print("Saved:", dirty_path, clean_path, ("and "+str(bad_rows_path) if not bad_rows.empty else "(no bad rows)"))


Nulls per column:
 user_id        0
signup_date    4
amount         1
notes          0
dtype: int64
Schema signature: 3335b2e19c5184dd20ee62bc95c56797
Saved: dirty.csv clean.csv and bad_rows.csv


In [ ]:
# preview
# Xem trước
display(df.head()) # Hiển thị 5 dòng đầu tiên của DataFrame đã làm sạch


,user_id,signup_date,amount,notes
0,101,2025-03-10 00:00:00+00:00,12.99,ok
1,102,NaT,15.00,"euro symbol, dot decimal"
2,103,NaT,1234.50,comma thousands
3,104,NaT,NaN,text should become NaN
4,105,NaT,-9.99,tz offset example


### Code Demo: Logging Bias


In [ ]:
import numpy as np, pandas as pd # Import thư viện numpy và pandas

# Step 1: Simulate events
# Bước 1: Mô phỏng các sự kiện
rng = np.random.default_rng(42) # Khởi tạo bộ tạo số ngẫu nhiên
n_users = 1000 # Số lượng người dùng
events = [] # Danh sách để lưu trữ các sự kiện
for u in range(n_users): # Duyệt qua từng người dùng
    events.append({"user":u, "event":"login"}) # Thêm sự kiện đăng nhập
    events.append({"user":u, "event":"logout"}) # Thêm sự kiện đăng xuất
df_true = pd.DataFrame(events) # Tạo DataFrame từ danh sách các sự kiện (dữ liệu "thật")

# Step 2: Apply logging bias (drop 30% of logouts)
# Bước 2: Áp dụng thiên kiến ghi nhật ký (loại bỏ 30% sự kiện đăng xuất)
mask = (df_true["event"]=="logout") # Tạo mask để xác định các dòng là sự kiện đăng xuất
drop_mask = rng.random(len(df_true)) < 0.3 # Tạo mask ngẫu nhiên để chọn 30% dòng để loại bỏ
df_obs = df_true[~(mask & drop_mask)].copy() # Áp dụng mask để loại bỏ các dòng đăng xuất đã chọn, tạo DataFrame "quan sát được"

# Step 3: Compare distributions
# Bước 3: So sánh các phân phối
true_dist = df_true["event"].value_counts(normalize=True) # Tính phân phối tỷ lệ của các loại sự kiện trong dữ liệu "thật"
obs_dist  = df_obs["event"].value_counts(normalize=True) # Tính phân phối tỷ lệ của các loại sự kiện trong dữ liệu "quan sát được"

print("True distribution:\n", true_dist) # In phân phối "thật"
print("\nObserved distribution (biased):\n", obs_dist) # In phân phối "quan sát được" (bị thiên kiến)


True distribution:
 event
login     0.5
logout    0.5
Name: proportion, dtype: float64

Observed distribution (biased):
 event
login     0.58309
logout    0.41691
Name: proportion, dtype: float64


### Code Demo: Data Drift

In [ ]:
from scipy.stats import entropy # Import hàm entropy từ scipy.stats

# 2019 spam filter training distribution
# Phân phối dữ liệu huấn luyện bộ lọc thư rác năm 2019
P = {"discount":0.40, "promo":0.05, "other":0.55}

# 2025 production distribution
# Phân phối dữ liệu trong môi trường sản xuất năm 2025
Q = {"discount":0.10, "promo":0.35, "other":0.55}

# Convert to aligned vectors
# Chuyển đổi sang vector có căn chỉnh
keys = sorted(set(P.keys()) | set(Q.keys())) # Lấy tất cả các khóa duy nhất từ cả P và Q, sắp xếp chúng
p = np.array([P[k] for k in keys]) # Tạo vector từ các giá trị trong P theo thứ tự của keys
q = np.array([Q[k] for k in keys]) # Tạo vector từ các giá trị trong Q theo thứ tự của keys

# Compute KL divergence D_KL(Q||P)
# Tính phân kỳ KL D_KL(Q||P)
dkl = entropy(q, p) # Sử dụng hàm entropy để tính phân kỳ KL giữa Q và P
print("D_KL(Q||P) =", dkl) # In kết quả phân kỳ KL
print("Keys:", keys) # In danh sách các khóa đã sử dụng để căn chỉnh


D_KL(Q||P) = 0.5424391160573705
Keys: ['discount', 'other', 'promo']


### Mini Demo: The Data Trust Pyramid

In [ ]:
import pandas as pd, numpy as np # Import thư viện pandas và numpy

# Raw data (messy)
# Dữ liệu thô (lộn xộn)
raw = pd.DataFrame({
    "user_id": ["001","002","003"], # ID người dùng
    "signup_date": ["03-10-25","2025/10/03","Oct 3 2025"], # Ngày đăng ký (các định dạng khác nhau)
    "amount": ["200USD","$150","one hundred"] # Số tiền (các định dạng khác nhau, có cả văn bản)
})
print("RAW DATA:\n", raw, "\n") # In dữ liệu thô

# Step 1: Clean & Normalize
# Bước 1: Làm sạch và Chuẩn hóa
df = raw.copy() # Tạo bản sao của dữ liệu thô để làm việc
# Chuyển đổi cột signup_date sang kiểu datetime, xử lý lỗi bằng cách thay thế bằng NaT, chuyển sang UTC
df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce", utc=True);
# Làm sạch cột amount để chỉ giữ lại số, dấu chấm, dấu gạch ngang, thay thế chuỗi rỗng bằng NaN, chuyển sang float
df["amount_num"] = (
    df["amount"].str.replace(r"[^0-9.]","",regex=True).replace("",np.nan).astype(float)
)
print("CLEANED & NORMALIZED:\n", df, "\n") # In dữ liệu đã làm sạch và chuẩn hóa

# Step 2: Log anomalies
# Bước 2: Ghi nhật ký các bất thường (anomalies)
logs = [] # Danh sách để lưu trữ các nhật ký lỗi
for idx,row in df.iterrows(): # Duyệt qua từng dòng của DataFrame
    if pd.isna(row["amount_num"]): # Kiểm tra xem giá trị amount_num có phải là NaN không (tức là không thể chuyển đổi sang số)
        logs.append(f"Row {idx}: amount invalid → {row['amount']}") # Thêm thông báo lỗi vào danh sách logs
print("LOGGED ANOMALIES:\n", logs, "\n") # In ra các nhật ký lỗi

# Step 3: Trust check (no NaNs allowed)
# Bước 3: Kiểm tra độ tin cậy (không cho phép có giá trị NaN)
trusted = df.dropna(subset=["signup_date","amount_num"]) # Loại bỏ các dòng có giá trị NaN trong cột signup_date hoặc amount_num
print("TRUSTED FOUNDATION:\n", trusted) # In ra dữ liệu "đáng tin cậy" (không có NaN ở các cột quan trọng)


RAW DATA:
   user_id signup_date       amount
0     001    03-10-25       200USD
1     002  2025/10/03         $150
2     003  Oct 3 2025  one hundred 

CLEANED & NORMALIZED:
   user_id               signup_date       amount  amount_num
0     001 2025-03-10 00:00:00+00:00       200USD       200.0
1     002 2025-10-03 00:00:00+00:00         $150       150.0
2     003 2025-10-03 00:00:00+00:00  one hundred         NaN 

LOGGED ANOMALIES:
 ['Row 2: amount invalid → one hundred'] 

TRUSTED FOUNDATION:
   user_id               signup_date  amount  amount_num
0     001 2025-03-10 00:00:00+00:00  200USD       200.0
1     002 2025-10-03 00:00:00+00:00    $150       150.0


/tmp/ipython-input-2050666915.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce", utc=True);
